### 使用AutoGen + Ollama 创建智能体

- 1.安装ollama:`ollama.ai`
- 2.使用ollama下载要使用的模型：
  - `ollama run mistral`
  - `ollama run codellama`
- 3.创建conda环境python>3.10
- 4.安装autogen:`pip install pyautogen`
- 5.安装litellama:`pip install litellm`
- 6.安装一些依赖包:
    - `pip install litellm[proxy]`
- 7.使用litellm加载ollama中下载的模型(**在代码中把http://0.0.0.0:4000改为http://127.0.0.1:4000，关闭vpn**):
   - `litellm --model ollama/mistral`
   - `litellm --model ollama/codellama`
     
  

In [1]:
import autogen

##### 配置大型语言模型（LLMs）的服务地址、API密钥及模型名称

In [2]:
config_list_mistral = [
    {
        'base_url': 'http://127.0.0.1:4000',
        'api_key': 'NULL',
        'model': "mistral"
    }
]

config_list_qwen = [
    {
        'base_url': 'http://127.0.0.1:45315',
        'api_key': 'NULL',
        'model': "qwen"
    }
]

##### 创建LLM配置对象

In [3]:
llm_config_mistral = {
    'config_list': config_list_mistral,
}

llm_config_qwen = {
    'config_list': config_list_qwen,
}

##### 初始化三个智能体：
- assistant：使用名为“mistral”的LLM模型的助理型智能体。
- coder：使用名为“codellama”的LLM模型的编程专家型智能体。
- user_proxy：代表用户的人工智能代理，可以接收用户的输入并在满足条件时终止对话。设置了参数如最大连续自动回复次数、终止消息判断函数（当消息内容以'TERMINATE'结尾时终止对话）、禁用了代码执行功能（或者指定了工作目录，这里注释掉了），并使用“mistral”模型进行文本生成。

In [4]:
assistant = autogen.AssistantAgent(
    name='Assistant',
    llm_config=llm_config_mistral,
)

poet = autogen.AssistantAgent(
    name='poet',
    llm_config=llm_config_qwen,
)

user_proxy = autogen.UserProxyAgent(
    name='user_proxy',
    human_input_mode='TERMINATE',
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get('content', '').rstrip().endswith('TERMINATE'),
    # code_execution_config={'woek_dir':'web'},
    code_execution_config=False,
    llm_config=llm_config_mistral,
    system_message="""Reply Terminate if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

##### 定义一个初始任务

In [5]:
task = '写一首关于人工智能的七言绝句'

##### 创建一个群聊实例

In [6]:
# 使用上述定义的三个智能体创建一个群聊，初始化消息列表为空，并设置了最多可进行12轮对话。
groupchat = autogen.GroupChat(agents=[user_proxy, poet, assistant], messages=[], max_round=12)

##### 创建群聊管理器

In [7]:
# 群聊管理器用于协调和管理整个对话过程，这里传入了之前创建的群聊实例和其中一个LLM配置。
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_mistral)

##### 开始对话

In [ ]:
user_proxy.initiate_chat(manager, message=task)

user_proxy (to chat_manager):

写一首关于人工智能的七言绝句

--------------------------------------------------------------------------------
